该部分代码是在Li_WISDM.ipynb基础上，调整时间窗与采样频率组合，做相应适应性分析。

从文件中读取数据，加载函数库

In [5]:
%matplotlib inline
import matplotlib.pylab as plt
import matplotlib
from tsfresh.examples.har_dataset import download_har_dataset, load_har_dataset, load_har_classes
# import seaborn as sns
from localtsfresh.tsfresh import extract_features, extract_relevant_features, select_features
from localtsfresh.tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters, EfficientFCParameters, MinimalFCParameters
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from pandas import DataFrame
import pandas as pd
import numpy as np
import os
import math

import logging

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from scipy.signal import butter, lfilter, lfilter_zi
from numpy import linalg as LA
from xgboost import XGBClassifier
import copy
from collections import Counter
from sklearn import neighbors
from sklearn.svm import SVC 

import datetime
import time

from multiprocessing import cpu_count
print(cpu_count())

# We set the logger to Error level
# This is not recommend for normal use as you can oversee important Warning messages
logging.basicConfig(level=logging.ERROR)

#根据Win，生成DataFrame格式中的Time列
def getTimeColumn(win):
    time=np.arange(win)
    for idx in range(1, sensornum):
        timetmp=np.arange(win)
        time=np.concatenate((time, timetmp), axis=0)
    time.shape=(len(time),1)
    return time

#根据Win和数据，生成DataFrame格式中的kind和value列
def getKindValueColumn(sd, win):
    for i in range(sensornum):
        kind=i*np.ones((win, 1),dtype=int)
        sensorcols=sd[i]
        sensorcols.shape=(win, 1)
        sdata=np.column_stack((kind, sensorcols))
        if i==0:
            sensorframe=sdata
        else:
            sensorframe=np.row_stack((sensorframe, sdata))
    return sensorframe

def getIdColumn(num, win):
    for i in range(num):
        if i==0:
            idary=0*np.ones((sensornum*win, 1))
        else:
            idary=np.row_stack((idary, i*np.ones((sensornum*win, 1))))
    return idary

def getDataLabelColumn(dataary,label, num, win):
    for i in range(filenum):
        if i==0:
            y=label[travel[i]][:num]
            data=dataary[travel[i]][:num*win*sensornum]
        else:
            y=np.row_stack((y, label[travel[i]][:num]))
            data=np.row_stack((data, dataary[travel[i]][:num*win*sensornum]))
    y.shape=(len(y),)
    y=pd.Series(y)
    return data, y


#读取所有数据，结果是字典，分别存储六种运动的传感器读数矩阵，每个矩阵的三列分别是time, kind, value
def loadData(win, step, fs, sampling):
    labeldict={}
    datadict={}
    files = os.listdir('/home/hadoop/data')
    if step<=win:
        for i in range(0, filenum):
            with open('/home/hadoop/data/%s' % files[i], 'r') as f:
                sensordata=np.loadtxt(f, delimiter=",")[::2, :]
                
                ########## feature extraction ###########
                sensordata = np.delete(sensordata, np.s_[:3], 1)
                sensordata[:, 2] = sensordata[:, 2] - 9.8
                sensordata = resample(sensordata, sampling)
                
                #mi = LA.norm(sensordata, 2, axis=1)
                #sma = np.sum(np.abs(sensordata), 1)
                #velo = calcVelo(sensordata[:,:2], fs, win)
                #sensordata = np.c_[sensordata, mi, sma]
                
                #loadDataPlot(sensordata, i)
                
                ########## filter ############
                #fltr2(sensordata, cutOff=15, fs=fs, order=5)
                #########################################
                
                max_num=int((len(sensordata)-win)/step)+1
                for j in range(0, max_num):
                    start_idx=step*j
                    end_idx=step*j+win
                    sd=sensordata[start_idx:end_idx,:]
                    sd=sd.transpose()
                        
                    time=getTimeColumn(win)
                    kindvalue=getKindValueColumn(sd, win)
                    travary=np.column_stack((time, kindvalue ))

                    if j == 0:
                        dataarray=travary
                    else:
                        dataarray=np.concatenate((dataarray, travary), axis=0)  

                labeldict[travel[i]]=i*np.ones((max_num, 1), dtype=int)
                datadict[travel[i]]=dataarray
                print(files[i]+" loaded!")
        use_num_max=filenum*max_num
        return datadict, labeldict, use_num_max
    else:
        raise IOError('\'step\' of slide window shoud be less than \'win\'')

#从所有数据dataary和其标签label中，选出总数为num的样本，每个类别选num/sensornum个样本。
def genTrainSample(dataary, label, num, win):
    idary=getIdColumn(num, win)
    labelnum=int(num/filenum) ;
    data, y=getDataLabelColumn(dataary, label, labelnum, win)
    data=np.column_stack((idary, data))
    dataframe = DataFrame(data, columns=['id', 'time', 'kind', 'value'])
    return dataframe, y


#画出第idx个样本3个传感器的数据
def plotSample(data, kind, idx, win, use_num):
    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(18.5, 10.5)
    tmp=data.iloc[:, -1].values
    for i in range(sensornum):
        plt.subplot(3,2,i+1)
        plt.title(sensor[i] + ' readings')
        begin=kind*win*use_num + idx*win*sensornum + win*i
        end=begin+win-1
        plt.plot(tmp[begin:end])
    plt.show()
    
def loadDataPlot(data, ind):
    plt.figure(ind)
    for i in xrange(0, data.shape[1]):
        fig = matplotlib.pyplot.gcf()
        fig.set_size_inches(18.5, 10.5)
        plt.subplot(3,2,i+1)
        plt.plot(data[:, i])
    return

################ Resampling ##################

def resample(data, sampling):
    tmp = []
    for i in xrange(0, 1):
        ind = np.arange(i, data.shape[0], sampling)
        tmp.append(data[ind, :])
    return np.concatenate(tmp, axis=0)

################ Calculate Velocity ###################

def calcVelo(acc, fs, win):
    velo = np.zeros(acc.shape)
    for i in xrange(0, acc.shape[0]):
        if i % win: 
            velo[i] = velo[i-1] + (acc[i-1]+acc[i])/(2*fs)
    return LA.norm(velo, 2, axis=1)
        

################ Butterworth 滤波 ###################

def butter_lowpass(cutOff, fs, order=5):
    nyq = 0.5 * fs
    normalCutoff = cutOff / nyq
    b, a = butter(order, normalCutoff, btype='low', analog = False)
    return b, a

def fltr(data, win, sensornum, cutOff, fs, order=5): #这个是在滑窗之后滤波的函数
    v = data.iloc[:, -1].values
    b, a = butter_lowpass(cutOff, fs, order)
    zi = np.tile(lfilter_zi(b, a), (sensornum,1))
    for i in xrange(0, v.size, win*sensornum):
        for j in xrange(0, sensornum):
            x = v[i+j*win: i+(j+1)*win]
            y, zi[j]= lfilter(b, a, x, zi=zi[j])
            data.iloc[i+j*win: i+(j+1)*win, -1] = y
    return

def fltr2(sensordata, cutOff, fs, order):  #这个是在数据读取后马上就滤波的函数
    b, a = butter_lowpass(cutOff, fs, order)
    for n in xrange(sensordata.shape[1]):
        zi = lfilter_zi(b, a)
        sensordata[:,n], zi = lfilter(b, a, sensordata[:,n], zi=zi)


4


经过XGBoost特征提取确定的13大类特征，因考虑不同窗口，采样频率的试验情况，并未采取后续有所缩减的8类特征。

In [6]:
extraction_settings = {'ar_coefficient': [{'coeff': 0, 'k': 10},
    {'coeff': 1, 'k': 10},
    {'coeff': 2, 'k': 10},
    {'coeff': 3, 'k': 10},
    {'coeff': 4, 'k': 10}],
          'longest_strike_above_mean': None,
          'longest_strike_below_mean': None,
          'mean_abs_change_quantiles': [{'qh': 0.2, 'ql': 0.0},
                                        {'qh': 0.4, 'ql': 0.0},
                                        {'qh': 0.6, 'ql': 0.0},
                                        {'qh': 0.8, 'ql': 0.0},
                                        {'qh': 1.0, 'ql': 0.0},
                                        {'qh': 0.4, 'ql': 0.2},
                                        {'qh': 0.6, 'ql': 0.2},
                                        {'qh': 0.8, 'ql': 0.2},
                                        {'qh': 1.0, 'ql': 0.2},
                                        {'qh': 0.6, 'ql': 0.4},
                                        {'qh': 0.8, 'ql': 0.4},
                                        {'qh': 1.0, 'ql': 0.4},
                                        {'qh': 0.8, 'ql': 0.6},
                                        {'qh': 1.0, 'ql': 0.6},
                                        {'qh': 1.0, 'ql': 0.8}],
          'autocorrelation': [{'lag': 0},{'lag': 1},{'lag': 2},{'lag': 3},{'lag': 4},
                              {'lag': 5},{'lag': 6},{'lag': 7},{'lag': 8},{'lag': 9}],
          'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
          'quantile': [{'q': 0.1},{'q': 0.2},{'q': 0.3},{'q': 0.4},
                       {'q': 0.6},{'q': 0.7},{'q': 0.8},{'q': 0.9}],
          'number_peaks': [{'n': 1}, {'n': 3}, {'n': 5}],
          'minimum': None,
          'maximum': None,
          'median': None,
          'sum_values': None,
          'spkt_welch_density': [{'coeff': 2}, {'coeff': 5}, {'coeff': 8}],
                    "fft_dc":None,
                    "fft_mean":None,
                    #"fft_var":None,
                    "fft_std":None,
                    "fft_kurt":None,
                    "fft_shape_mean":None,
                    "fft_shape_std":None,
                    "fft_shape_skew":None,
                    "fft_shape_kurt":None
                      }

时间窗口，（降）采样频率的设定

In [7]:
#可修改常量
# 六种出行方式
travel=["bus", "car", "running", "stationary", "subway", "walking"]
# 六个传感器数据
sensor=["azimath", "pitch", "roll"]
# 一些常量
filenum=6
#窗口步长大小
#win = [100，200,300,400,500,600]/sample,注意win是在采样间隔基础上算的
win=600
#步长为时间窗口的一半
step=math.ceil(win/2)
#sampling[1,2,3,4,5,10,20] ,分别对应100hz，50hz，33hz,25hz，20hz,10hz,5hz
sampling=1  #采样间隔

sensornum=len(sensor)
data, label, use_num_max = loadData(win, step, 0, sampling)

print("\n \033[1;33;48m NOTE: use_num should be less than %s"%(use_num_max))

bus_data_set_101 loaded!
car_data_set_103 loaded!
running_data_set_8 loaded!
stationary_data_set_3 loaded!
subway_data_set_102 loaded!
walking_data_set_7 loaded!

  NOTE: use_num should be less than 1206


In [8]:
#使用的样本数量(一个窗口的数据是一个样本)，建议设置陈6的倍数
use_num =1206
#取出use_num个样本，并将data转化成tsfresh需要的pandas.DataFrame类型，存储为df；将label转化为特征过滤需要的pandas.Series类型，存储为y
df, y=genTrainSample(data, label, use_num, win)
master_df = df
print(master_df.shape)

(2170800, 4)


总特征提取时间

In [9]:
# extraction_settings = ComprehensiveFCParameters()
#extraction_settings = EfficientFCParameters()
# extraction_settings = MinimalFCParameters()
#extraction_settings.update({"fft_dc":None,"fft_mean":None,"fft_var":None,"fft_std":None,"fft_kurt":None,"fft_shape_mean":None,"fft_shape_std":None,"fft_shape_skew":None,"fft_shape_kurt":None})
%time X = extract_features(master_df, default_fc_parameters=extraction_settings, column_id='id', column_sort="time", column_kind="kind", column_value="value");
#%time X = extract_features(master_df, column_id='id', column_sort="time", column_kind="kind", column_value="value");

It is ok!


Feature Extraction: 100%|██████████| 3618/3618 [01:35<00:00, 38.00it/s]


CPU times: user 2.28 s, sys: 198 ms, total: 2.48 s
Wall time: 1min 36s


多次重复计算100个窗口特征时间，因为timeit魔术命令总是取三次作为均值，后续并未采用这一数据

In [10]:
%timeit Y = extract_features(master_df.ix[0:(win*3*100-1)], default_fc_parameters=extraction_settings, column_id='id', column_sort="time", column_kind="kind", column_value="value");

It is ok!


Feature Extraction: 100%|██████████| 300/300 [00:08<00:00, 36.06it/s]

It is ok!



Feature Extraction: 100%|██████████| 300/300 [00:08<00:00, 35.29it/s]

It is ok!



Feature Extraction: 100%|██████████| 300/300 [00:07<00:00, 40.23it/s]

It is ok!



Feature Extraction: 100%|██████████| 300/300 [00:08<00:00, 35.86it/s]

1 loop, best of 3: 7.58 s per loop


100个窗口cpu计算时间，重复多次取均值

In [11]:
##100次窗口计算时间
cpu_time = 0 #cpu时间
for i in xrange(0,5):
    start = time.clock()
    Y = extract_features(master_df.ix[0:(win*3*100-1)], default_fc_parameters=extraction_settings, column_id='id', column_sort="time", column_kind="kind", column_value="value");
    end = time.clock()
    cpu_time +=end-start
print('100 window CPU time for extract_features is %f s '%(cpu_time/5))

It is ok!


Feature Extraction: 100%|██████████| 300/300 [00:08<00:00, 35.57it/s]

It is ok!



Feature Extraction: 100%|██████████| 300/300 [00:08<00:00, 35.65it/s]


It is ok!


...

It is ok!



Feature Extraction: 100%|██████████| 300/300 [00:07<00:00, 39.77it/s]

It is ok!



Feature Extraction: 100%|██████████| 300/300 [00:08<00:00, 35.98it/s]

100 window CPU time for extract_features is 0.296000 s 


100个窗口程序执行时间，考虑了IO时间。重复多次取均值

In [12]:
execute_time = 0 #程序执行时间
for i in xrange(0,5):
    start = datetime.datetime.now()
    Y = extract_features(master_df.ix[0:(win*3*100-1)], default_fc_parameters=extraction_settings, column_id='id', column_sort="time", column_kind="kind", column_value="value");
    end = datetime.datetime.now()
    execute_time+= (end-start).seconds
print('100 window execute time for extract_features is %f s'%(execute_time/5))

It is ok!


Feature Extraction: 100%|██████████| 300/300 [00:07<00:00, 38.80it/s]


It is ok!


Feature Extraction: 100%|██████████| 300/300 [00:07<00:00, 40.35it/s]


It is ok!


...

It is ok!



Feature Extraction: 100%|██████████| 300/300 [00:08<00:00, 35.05it/s]


It is ok!


Feature Extraction: 100%|██████████| 300/300 [00:08<00:00, 36.72it/s]

100 window execute time for extract_features is 7.000000 s


In [14]:
impute(X)
X.shape

(1206, 183)

数据的归一化

In [15]:
for n in X.columns:
    std = np.std(X[n])
    X[n] = (X[n]-np.mean(X[n]))/std if std>0 else X[n]-np.mean(X[n])

# 分类XGBoost,Random Forest,Decision Tree,KNN

XGBoost20次随机划分数据集，计算均值及平均混淆矩阵

In [16]:
##XGBoost:重复20次，准确率及混淆矩阵
mean = 0
for i in xrange(0, 20):
    X_train_tmp, X_test, y_train_tmp, y_test = train_test_split(X, y, test_size=.2)
    X_train, X_val, y_train, y_val = train_test_split(X_train_tmp, y_train_tmp, test_size=.125)
    eval_set = [(X_val, y_val)]
    cl = XGBClassifier(max_depth=5, n_estimators=200, objective='multi:softmax', reg_lambda=2)
    cl.fit(X_train, y_train, eval_metric='mlogloss', eval_set=eval_set, early_stopping_rounds=30, verbose=False)
    score = accuracy_score(y_test, cl.predict(X_test))
    tempcross=pd.crosstab(cl.predict(X_test), y_test, rownames=['actual'], colnames=['preds'])
    print('Loop %d, accuracy=%f' % (i, score))
    mean += score
    if i==0:
        crosstab = tempcross
    else:
        crosstab = crosstab+tempcross
print('Mean accuracy is %f' % (mean/20))
print(crosstab/20)

Loop 0, accuracy=0.942149
Loop 1, accuracy=0.933884
Loop 2, accuracy=0.921488
Loop 3, accuracy=0.942149
Loop 4, accuracy=0.929752
Loop 5, accuracy=0.925620
Loop 6, accuracy=0.938017
Loop 7, accuracy=0.929752
Loop 8, accuracy=0.942149
Loop 9, accuracy=0.946281
Loop 10, accuracy=0.904959
Loop 11, accuracy=0.933884
Loop 12, accuracy=0.933884
Loop 13, accuracy=0.962810
Loop 14, accuracy=0.917355
Loop 15, accuracy=0.942149
Loop 16, accuracy=0.933884
Loop 17, accuracy=0.933884
Loop 18, accuracy=0.921488
Loop 19, accuracy=0.925620
Mean accuracy is 0.933058
preds       0      1      2      3     4      5
actual                                         
0       34.85   2.60   0.25   0.15   1.3   0.25
1        1.45  33.15   0.70   0.15   1.5   0.00
2        0.00   0.00  38.75   0.00   0.0   0.05
3        0.00   0.35   0.00  40.70   0.1   0.00
4        3.60   2.30   0.20   0.45  36.8   0.00
5        0.05   0.20   0.45   0.00   0.1  41.55


选取随机森林，决策树，KNN三种分类器，同样随机划分20次，取精度均值

In [17]:
mean1 = 0
mean2 = 0
mean3 = 0
for i in xrange(0, 20):
    X_train_tmp, X_test, y_train_tmp, y_test = train_test_split(X, y, test_size=.2)
    X_train, X_val, y_train, y_val = train_test_split(X_train_tmp, y_train_tmp, test_size=.125)
    cl1 = RandomForestClassifier()
    cl1.fit(X_train, y_train)
    score1 = accuracy_score(y_test, cl1.predict(X_test))
    cl2 = DecisionTreeClassifier()
    cl2.fit(X_train, y_train)
    score2 = accuracy_score(y_test, cl2.predict(X_test))
    cl3 = neighbors.KNeighborsClassifier()  
    cl3.fit(X_train, y_train)
    score3 = accuracy_score(y_test, cl3.predict(X_test))
    print('Loop %d, accuracy=%f,%f,%f' % (i, score1,score2,score3))
    mean1 += score1
    mean2 += score2
    mean3 += score3
print('RF Mean accuracy is %f' % (mean1/20))
print('DT Mean accuracy is %f' % (mean2/20))
print('KNN Mean accuracy is %f' % (mean3/20))

Loop 0, accuracy=0.913223,0.871901,0.900826
Loop 1, accuracy=0.921488,0.876033,0.867769
Loop 2, accuracy=0.913223,0.851240,0.867769
Loop 3, accuracy=0.888430,0.855372,0.855372
Loop 4, accuracy=0.913223,0.830579,0.900826
Loop 5, accuracy=0.896694,0.851240,0.892562
Loop 6, accuracy=0.925620,0.904959,0.871901
Loop 7, accuracy=0.921488,0.909091,0.896694
Loop 8, accuracy=0.855372,0.855372,0.888430
Loop 9, accuracy=0.900826,0.859504,0.884298
Loop 10, accuracy=0.867769,0.855372,0.900826
Loop 11, accuracy=0.896694,0.855372,0.892562
Loop 12, accuracy=0.896694,0.826446,0.863636
Loop 13, accuracy=0.917355,0.896694,0.871901
Loop 14, accuracy=0.876033,0.859504,0.851240
Loop 15, accuracy=0.909091,0.884298,0.904959
Loop 16, accuracy=0.946281,0.859504,0.880165
Loop 17, accuracy=0.900826,0.880165,0.863636
Loop 18, accuracy=0.938017,0.834711,0.838843
Loop 19, accuracy=0.925620,0.851240,0.876033
RF Mean accuracy is 0.906198
DT Mean accuracy is 0.863430
KNN Mean accuracy is 0.878512


评估100个窗口多次预测的平均时间

In [18]:
%timeit cl.predict(X_test.iloc[0:99])
%timeit cl1.predict(X_test.iloc[0:99])
%timeit cl2.predict(X_test.iloc[0:99])
%timeit cl3.predict(X_test.iloc[0:99])

100 loops, best of 3: 4.55 ms per loop
1000 loops, best of 3: 963 µs per loop
1000 loops, best of 3: 204 µs per loop
10 loops, best of 3: 20.8 ms per loop
